In [90]:
import pandas as pd
import regex as re
import string
import json
import csv

In [42]:
# Question ID and Answer text pair
df = pd.read_csv("train/FiQA_train_doc_final.tsv", sep="\t")

df.head(5)

,Unnamed: 0,docid,doc,timestamp
0,0,3,I'm not saying I don't like the idea of on-the...,Oct 03 '12 at 14:56
1,1,31,So nothing preventing false ratings besides ad...,Sep 01 '17 at 13:36
2,2,56,You can never use a health FSA for individual ...,Jun 9 '14 at 17:37
3,3,59,Samsung created the LCD and other flat screen ...,Dec 27 at 01:37
4,4,63,Here are the SEC requirements: The federal sec...,Jul 14 '14 at 8:10


In [82]:
collection = df[['docid', 'doc']]
collection = collection.sort_values(by=['docid'])

def pre_process(doc):
    
    doc = str(doc)
    
    x = re.sub('[“”$%!&"@#()\-\*\+,/:;<=>?@[\]\^_`{\}~]', ' ', doc)
    
    y = re.sub('[\.\']', "", x)
    
    z = y.lower()
    
    return z

# def hyphen(doc):
    
#     doc = str(doc)
    
#     x = re.sub('\-', ' ', doc)
    
#     return x

# def remove_period(doc):
    
#     doc = str(doc)
    
#     pattern = re.compile(r'\.(\s[A-Z])')
    
#     x = pattern.sub(r'\1', doc)
    
#     if x[-1] == ".":
#         x = x[:-1]
        
#     return x

collection['doc'] = collection['doc'].apply(pre_process)

#collection['doc'] = collection['doc'].apply(hyphen)

#collection['doc'] = collection['doc'].apply(remove_period)

#collection['doc'] = collection['doc'].str.lower()

a = collection['doc'][0]

print(a)

collection.head(10)

im not saying i dont like the idea of on the job training too  but you cant expect the company to do that training workers is not their job   theyre building software perhaps educational systems in the us  or their students  should worry a little about getting marketable skills in exchange for their massive investment in education  rather than getting out with thousands in student debt and then complaining that they arent qualified to do anything


,docid,doc
0,3,im not saying i dont like the idea of on the j...
1,31,so nothing preventing false ratings besides ad...
2,56,you can never use a health fsa for individual ...
3,59,samsung created the lcd and other flat screen ...
4,63,here are the sec requirements the federal sec...
5,100,only relevant to those with fantasy economy te...
6,108,futures contracts are a member of a larger cla...
7,125,this month when you join scentsy you get a fre...
8,132,whenever you pay or withdraw some fund from yo...
9,138,so you asked him in 2010 how he was gong to co...


In [60]:
print(len(collection))

57638


In [85]:
with open("fiqa-passage/collection.tsv",'w') as write_tsv:
    write_tsv.write(collection.to_csv(sep='\t', index=False, header=False))

In [86]:
output_jsonl_file = open("fiqa-passage/collection_jsonl/docs00.json", 'w', encoding='utf-8', newline='\n')

with open("fiqa-passage/collection.tsv", encoding='utf-8') as f:
    for i, line in enumerate(f):
        id_text = line.rstrip().split('\t')
        doc_id, doc_text = id_text[0], id_text[1]
        
        output_dict = {'id': doc_id, 'contents': doc_text}
        
        output_jsonl_file.write(json.dumps(output_dict) + '\n')

In [68]:
# Question ID and Question text pair
query_df = pd.read_csv("train/FiQA_train_question_final.tsv", sep="\t")

query_df.head(5)

,Unnamed: 0,qid,question,timestamp
0,0,0,What is considered a business expense on a bus...,Nov 8 '11 at 15:14
1,1,1,Claiming business expenses for a business with...,May 13 '14 at 13:17
2,2,2,Transferring money from One business checking ...,Jan 20 '16 at 20:31
3,3,3,Having a separate bank account for business/in...,Mar 1 at 0:24
4,4,4,Business Expense - Car Insurance Deductible Fo...,Mar 4 at 0:26


In [83]:
queries = query_df[['qid', 'question']]

queries['question'] = queries['question'].apply(pre_process)

queries.head(5)

/home/bithiah/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,qid,question
0,0,what is considered a business expense on a bus...
1,1,claiming business expenses for a business with...
2,2,transferring money from one business checking ...
3,3,having a separate bank account for business in...
4,4,business expense car insurance deductible fo...


In [84]:
print(queries)

        qid                                           question
0         0  what is considered a business expense on a bus...
1         1  claiming business expenses for a business with...
2         2  transferring money from one business checking ...
3         3  having a separate bank account for business in...
4         4  business expense   car insurance deductible fo...
...     ...                                                ...
6643  11092  is real  physical  money traded during online ...
6644  11096              pensions  annuities  and  retirement 
6645  11097  what does it mean to a life insurance policy h...
6646  11099  can capital loss in traditional ira and roth i...
6647  11104  selling a stock for gain to offset other stock...

[6648 rows x 2 columns]


In [101]:
with open("fiqa-passage/queries_train.tsv",'w') as write_tsv:
    write_tsv.write(queries.to_csv(sep='\t', index=False, header=False))

In [106]:
# Question ID and Answer ID pair
qid_docid = pd.read_csv("train/FiQA_train_question_doc_final.tsv", sep="\t")

qid_docid = qid_docid [['qid', 'docid']]

qid_docid.head(5)

,qid,docid
0,0,18850
1,1,14255
2,2,308938
3,3,296717
4,3,100764


In [107]:
qid_docid = qid_docid.assign(new1=0)

qid_docid = qid_docid.assign(new2=1)

qid_docid.head(5)

,qid,docid,new1,new2
0,0,18850,0,1
1,1,14255,0,1
2,2,308938,0,1
3,3,296717,0,1
4,3,100764,0,1


In [110]:
qrels_train = qid_docid[['qid', 'new1', 'docid', 'new2']]

qrels_train.head(5)

,qid,new1,docid,new2
0,0,0,18850,1
1,1,0,14255,1
2,2,0,308938,1
3,3,0,296717,1
4,3,0,100764,1


In [114]:
print(len(qrels_train))

print("Number of unique questions: ")
print(qrels_train['qid'].nunique())

print("Number of unique passages: ")
print(qrels_train['docid'].nunique())

17110
Number of unique questions: 
6648
Number of unique passages: 
17110


In [115]:
with open("fiqa-passage/qrels_train.tsv",'w') as write_tsv:
    write_tsv.write(qrels_train.to_csv(sep='\t', index=False, header=False))

In [121]:
rel = qid_docid['qid'] == 0

rel_df = qid_docid[rel]

rel_df.head()

,qid,docid,new1,new2
0,0,18850,0,1


In [134]:
print(queries.at[0, 'question'])

x = collection[collection['docid']==18850]

print()

print(x.at[1800, 'doc'])

what is considered a business expense on a business trip 

the irs guidance pertaining to the subject  in general the best i can say is your business expense may be deductible  but it depends on the circumstances and what it is you want to deduct travel taxpayers who travel away from home on business may deduct related   expenses  including the cost of reaching their destination  the cost   of lodging and meals and other ordinary and necessary expenses   taxpayers are considered  traveling away from home  if their duties   require them to be away from home substantially longer than an   ordinary day’s work and they need to sleep or rest to meet the demands   of their work the actual cost of meals and incidental expenses may be   deducted or the taxpayer may use a standard meal allowance and reduced   record keeping requirements regardless of the method used  meal   deductions are generally limited to 50 percent as stated earlier    only actual costs for lodging may be claimed as an exp

In [136]:
x = collection[collection['docid']==531578]

print()

print(x.at[51106, 'doc'])


it depends on what the  true  reason for the trip is  if you decide to deduct the trip as a business expense  then during an audit you will be asked why you had to go there if there was nothing accomplished via the travel  that is  you worked from the hotel  met with no clients  visited no tradeshows  etc  then the expense is unlikely to be allowed yes  on a business trip you can do sightseeing if you wish  though you cant deduct any sightseeing specific expenses  like admission to a tourist attraction   but if you are just working while on vacation  then the trip itself is not deductible  since there was no business benefit to traveling in the first place
